# Task 1.
$ Y $ - will use XAI

$ \hat{Y} $ - enrolled in training

### Demographic parity
$ P(\hat{Y} | Blue) / P(\hat{Y} | Red) = 0.65 / 0.5 = 1.3 $

### Equal opportunity
$ P(\hat{Y} | Blue, Y) / P(\hat{Y} | Red, Y) = 0.75 / 0.5 = 1.5 $

### Predictive rate parity
$ P(Y | \hat{Y},  Blue) / P(Y | \hat{Y}, Red) = \frac{12}{13} / \frac{1}{2} = 24 / 13 = 1.83 $



# Task 2.

## 2.

| Model                          | Statistical parity | Equal opportunity | Predictive parity |
|--------------------------------|--------------------|-------------------|-------------------|
| Random Forest                  | 0.298182           | 0.922492          | 1.047222          |

This model has statistical parity close to 0.3, which means that it predicts higher income for males.
Other coefficients are in the desired [0.8, 1.25] interval.

## 3.

| Model                          | Statistical parity | Equal opportunity | Predictive parity |
|--------------------------------|--------------------|-------------------|-------------------|
| Logistic Regression            | 0.533333           | 1.143396          | 0.722368          |

Here are the fairness coefficients results for Logistic Regression model. This model is much simpler (and worse in terms of metrics - AUC).
It is probably better - Statistical parity is 0.53 instead of 0.3. Predictive parity is worse than earlier (0.72).

## 4.

| Model                          | Statistical parity | Equal opportunity | Predictive parity |
|--------------------------------|--------------------|-------------------|-------------------|
| Random Forest, bias mitigated  | 0.380000           | 1.056180          | 0.941176          |

Here we have the Random Forest model with the ROC pivot bias mitigation technique. As expected, this model is more fair than the standard Random Forest, having 0.38 Statistical parity coefficient.


## 5.

| Model                          | Statistical parity | Equal opportunity | Predictive parity | AUC   |
|--------------------------------|--------------------|-------------------|-------------------|-------|
| Random Forest                  | 0.298182           | 0.922492          | 1.047222          | 0.785 |
| Logistic Regression            | 0.533333           | 1.143396          | 0.722368          | 0.618 |
| Random Forest, bias mitigated  | 0.380000           | 1.056180          | 0.941176          | 0.779 |

Here is the comparison between all the 3 models with their metric (AUROC) and their fairness coefficients.
As expected, making the model more fair results in a bit worse auc. It is still of course worth it in most of the cases.



# Appendix

## 0.
Here the data is loaded (same as in previous homework) and a simple model is trained and evaluated. It is Random Forest Classifier from sklearn with default parameters.

Loading and preparing the data consists of:
- one hot encoding (models like logistic regression require this)
- splitting between target (y) and x

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
import dalex as dx


np.random.seed(42)


def load_data():
    df = pd.read_csv('adult.csv')

    # One hot encoding (for linear classifier)
    df = pd.get_dummies(df, columns=["workclass", "education", "marital-status", "occupation", "relationship", "race", "gender", "native-country"])

    # Get targets
    y_all = df['income'] == ">50K"

    x_all = df.drop(columns=['income'])

    # Split data to train and test
    return train_test_split(x_all, y_all, test_size=0.2, random_state=42)


x_train, x_test, y_train, y_test = load_data()
print(f"{x_train.shape=}", f"{x_test.shape=}", f"{y_train.shape=}", f"{y_test.shape=}")

x_train.shape=(39073, 108) x_test.shape=(9769, 108) y_train.shape=(39073,) y_test.shape=(9769,)


# 1.
Training the models

In [2]:
def get_model(verbose=False, model=None, n_estimators=100, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test):
    if model is None:
        model = RandomForestClassifier(n_estimators=n_estimators)
    metrics = {
        "auc": roc_auc_score,
        "accuracy": accuracy_score
    }
    model.fit(x_train, y_train)
    pred_test = model.predict(x_test)
    if verbose:
        print({metric_name: metric_fun(y_test, pred_test) for metric_name, metric_fun in metrics.items()})

    return model

model = get_model(True)
model_linear = get_model(True, LogisticRegression())

{'auc': 0.7850466604892535, 'accuracy': 0.8603746545193981}
{'auc': 0.6186399356334563, 'accuracy': 0.8036646534957519}


## 2 & 3.

In [3]:
def pf_xgboost_classifier_categorical(model, df):
    df.loc[:, df.dtypes == 'object'] =\
        df.select_dtypes(['object'])\
        .apply(lambda x: x.astype('category'))
    return model.predict_proba(df)[:, 1]

protected = x_test['gender_Male'].apply(lambda x: "male" if x else "female")
privileged = "male"

def stats(_model):
    explainer = dx.Explainer(_model, x_test, y_test, predict_function=pf_xgboost_classifier_categorical, verbose=False)

    fobject = explainer.model_fairness(
        protected=protected,
        privileged=privileged,
    )

    print(_model.__class__.__name__)
    print(fobject.result[['STP', 'TPR', 'PPV']].iloc[0])
    # print(fobject.result)
    # fobject.plot()
    # fobject.fairness_check()

stats(model)
stats(model_linear)

RandomForestClassifier
STP    0.298182
TPR    0.922492
PPV    1.047222
Name: female, dtype: float64
LogisticRegression
STP    0.533333
TPR    1.143396
PPV    0.722368
Name: female, dtype: float64


## 3.

In [6]:
from copy import copy
from dalex.fairness import roc_pivot

X_train_without_prot, X_test_without_prot = x_train.drop("gender_Male", axis=1).drop("gender_Female", axis=1), x_test.drop("gender_Male", axis=1).drop("gender_Female", axis=1)

model_without_prot = get_model(x_train=X_train_without_prot, x_test=X_test_without_prot, verbose=True)

explainer_without_prot = dx.Explainer(
    model_without_prot,
    X_test_without_prot,
    y_test,
    predict_function=pf_xgboost_classifier_categorical,
    label="XGBClassifier without the protected attribute",
    verbose=False
)

fobject_without_prot = explainer_without_prot.model_fairness(protected, privileged)

# roc_pivot
explainer_roc_pivot = roc_pivot(
    copy(explainer_without_prot),
    protected,
    privileged,
    verbose=False
)

print(explainer_roc_pivot.model_fairness(protected, privileged).result[['STP', 'TPR', 'PPV']].iloc[0])

{'auc': 0.7794231417109099, 'accuracy': 0.8570989865902344}
STP    0.380000
TPR    1.056180
PPV    0.941176
Name: female, dtype: float64
